In [2]:
!pip install easyocr opencv-python pillow jiwer matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 121.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.2/978.2 kB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 30.5 MB/s eta 0:00:00


In [3]:
!ls


sample_data


In [4]:
!ls ~/.kaggle


ls: cannot access '/root/.kaggle': No such file or directory


In [5]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"vikingdoomguy","key":"52351fe11f5f44627e584382baee0e6f"}'}

In [6]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [7]:
!kaggle --version


Kaggle API 1.7.4.5


In [8]:
!kaggle datasets download -d senju14/ocr-dataset-of-multi-type-documents


Dataset URL: https://www.kaggle.com/datasets/senju14/ocr-dataset-of-multi-type-documents
License(s): MIT
 99% 5.10G/5.16G [01:36<00:03, 17.0MB/s]
100% 5.16G/5.16G [01:36<00:00, 57.3MB/s]


In [9]:
!ls *.zip


ocr-dataset-of-multi-type-documents.zip


In [10]:
!unzip ocr-dataset-of-multi-type-documents.zip


Streaming output truncated to the last 5000 lines.
  inflating: invoice/test/annotations/X51006556596.json  
  inflating: invoice/test/annotations/X51006556648.json  
  inflating: invoice/test/annotations/X51006556810.json  
  inflating: invoice/test/annotations/X51006556831.json  
  inflating: invoice/test/annotations/X51006557173.json  
  inflating: invoice/test/annotations/X51006557185.json  
  inflating: invoice/test/annotations/X51006557208.json  
  inflating: invoice/test/annotations/X51006557508.json  
  inflating: invoice/test/annotations/X51006619507.json  
  inflating: invoice/test/annotations/X51006619564.json  
  inflating: invoice/test/annotations/X51006619570.json  
  inflating: invoice/test/annotations/X51006619704.json  
  inflating: invoice/test/annotations/X51006619765.json  
  inflating: invoice/test/annotations/X51006619784.json  
  inflating: invoice/test/annotations/X51006620192.json  
  inflating: invoice/test/annotations/X51006647932.json  
  inflating: invoice/

In [11]:
!ls


document  invoice      ocr-dataset-of-multi-type-documents.zip	sample_data
form	  kaggle.json  real_life


In [12]:
import easyocr
import cv2
import matplotlib.pyplot as plt


In [13]:
reader = easyocr.Reader(['en'], gpu=True)


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [14]:
img_path = "form/test/images/91939637.png"

results = reader.readtext(img_path)

for bbox, text, conf in results:
    print(f"{text}  (confidence: {conf:.2f})")


TO:  (confidence: 0.75)
DATE  (confidence: 1.00)
FROM:  (confidence: 0.58)
true  (confidence: 0.55)
S3.00 CARTON COUPON PROMOTION  (confidence: 0.94)
PROMOTIONAL PERIOD:  (confidence: 0.84)
May-June 1992  (confidence: 0.84)
CONSUMER OFFER;  (confidence: 0.75)
33 Off Store Redeemabla Carton Coupon  (confidence: 0.34)
DISPLAY BRANDS:  (confidence: 0.99)
TRUE (AII Packings)  (confidence: 0.79)
DISPLAY AGREEMENT:  (confidence: 1.00)
Lorillard Sales Representative wll supply all display materals and  (confidence: 0.79)
assemble display: Display containing deals wll be placed on  (confidence: 0.77)
sell-servica basis Ior  (confidence: 0.51)
two-week percd_  (confidence: 0.44)
AUTHORIZATION FOR PLACEMENT  (confidence: 0.71)
TYPE DISPLAY  (confidence: 0.63)
NO  (confidence: 0.50)
CARIONS  (confidence: 0.96)
COUNTER  (confidence: 1.00)
FLOOR  (confidence: 1.00)
POSTER  (confidence: 1.00)
(~  (confidence: 0.24)
approved)  (confidence: 1.00)
DATE; FROM  (confidence: 0.67)
TO  (confidence: 0.31)
S

In [15]:
easyocr_text = " ".join([text for (_, text, _) in results])
print(easyocr_text)


TO: DATE FROM: true S3.00 CARTON COUPON PROMOTION PROMOTIONAL PERIOD: May-June 1992 CONSUMER OFFER; 33 Off Store Redeemabla Carton Coupon DISPLAY BRANDS: TRUE (AII Packings) DISPLAY AGREEMENT: Lorillard Sales Representative wll supply all display materals and assemble display: Display containing deals wll be placed on sell-servica basis Ior two-week percd_ AUTHORIZATION FOR PLACEMENT TYPE DISPLAY NO CARIONS COUNTER FLOOR POSTER (~ approved) DATE; FROM TO STORES PARTICIPATING: THE FOLLOWING QUANTITY OF TRUE CARTONS MUST BE AVAILABLE FOR THE LORILLARD SALES REPRESENTATIVE TO ASSEMBLE THE DISPLAY ON THE DATE INDICATED ABOVE, TRUE NUMBER NUMBER PACKING OF CARIONS PACKING OF_ CARTONS FILTER KS; MENTHOL KS: FILTER 100's MENTKOL 100's BOX 80's BOX 100s MCRLAMMUS MHERE AVALABE) MERCHANDISE: WILL BE ARBITRARILY SHIPPED TO STORE ORDER, THROUGH YOUR SUPPLER: Authorized by: Date: 1


In [17]:
ground_truth_clean = "COSENTINOS Market DOWNTOWN"
print(ground_truth_clean)


COSENTINOS Market DOWNTOWN


In [18]:
from jiwer import wer, cer

# clean EasyOCR output (remove extra spaces/newlines)
easyocr_text_clean = " ".join(easyocr_text.split())
ground_truth_clean = " ".join(ground_truth_clean.split())

print("Ground Truth:")
print(ground_truth_clean)

print("\nEasyOCR OCR Output:")
print(easyocr_text_clean)

print("\nEasyOCR Word Error Rate (WER):", wer(ground_truth_clean, easyocr_text_clean))
print("EasyOCR Character Error Rate (CER):", cer(ground_truth_clean, easyocr_text_clean))


Ground Truth:
COSENTINOS Market DOWNTOWN

EasyOCR OCR Output:
TO: DATE FROM: true S3.00 CARTON COUPON PROMOTION PROMOTIONAL PERIOD: May-June 1992 CONSUMER OFFER; 33 Off Store Redeemabla Carton Coupon DISPLAY BRANDS: TRUE (AII Packings) DISPLAY AGREEMENT: Lorillard Sales Representative wll supply all display materals and assemble display: Display containing deals wll be placed on sell-servica basis Ior two-week percd_ AUTHORIZATION FOR PLACEMENT TYPE DISPLAY NO CARIONS COUNTER FLOOR POSTER (~ approved) DATE; FROM TO STORES PARTICIPATING: THE FOLLOWING QUANTITY OF TRUE CARTONS MUST BE AVAILABLE FOR THE LORILLARD SALES REPRESENTATIVE TO ASSEMBLE THE DISPLAY ON THE DATE INDICATED ABOVE, TRUE NUMBER NUMBER PACKING OF CARIONS PACKING OF_ CARTONS FILTER KS; MENTHOL KS: FILTER 100's MENTKOL 100's BOX 80's BOX 100s MCRLAMMUS MHERE AVALABE) MERCHANDISE: WILL BE ARBITRARILY SHIPPED TO STORE ORDER, THROUGH YOUR SUPPLER: Authorized by: Date: 1

EasyOCR Word Error Rate (WER): 43.0
EasyOCR Character 

In [19]:
import os

os.makedirs("easyocr_train/images", exist_ok=True)
os.makedirs("easyocr_train/labels", exist_ok=True)


In [20]:
ls form/train/images | head -n 20 | xargs -I {} cp form/train/images/{} easyocr_train/images/


In [21]:
with open("easyocr_train/labels/train.txt", "w") as f:
    f.write("COSENTINOS Market DOWNTOWN\n")


In [22]:
reader_finetuned = easyocr.Reader(
    ['en'],
    gpu=True,
    recognizer=True
)


In [23]:
results_ft = reader_finetuned.readtext("form/test/images/91939637.png")

easyocr_ft_text = " ".join([text for (_, text, _) in results_ft])
easyocr_ft_text_clean = " ".join(easyocr_ft_text.split())

print(easyocr_ft_text_clean)


TO: DATE FROM: true S3.00 CARTON COUPON PROMOTION PROMOTIONAL PERIOD: May-June 1992 CONSUMER OFFER; 33 Off Store Redeemabla Carton Coupon DISPLAY BRANDS: TRUE (AII Packings) DISPLAY AGREEMENT: Lorillard Sales Representative wll supply all display materals and assemble display: Display containing deals wll be placed on sell-servica basis Ior two-week percd_ AUTHORIZATION FOR PLACEMENT TYPE DISPLAY NO CARIONS COUNTER FLOOR POSTER (~ approved) DATE; FROM TO STORES PARTICIPATING: THE FOLLOWING QUANTITY OF TRUE CARTONS MUST BE AVAILABLE FOR THE LORILLARD SALES REPRESENTATIVE TO ASSEMBLE THE DISPLAY ON THE DATE INDICATED ABOVE, TRUE NUMBER NUMBER PACKING OF CARIONS PACKING OF_ CARTONS FILTER KS; MENTHOL KS: FILTER 100's MENTKOL 100's BOX 80's BOX 100s MCRLAMMUS MHERE AVALABE) MERCHANDISE: WILL BE ARBITRARILY SHIPPED TO STORE ORDER, THROUGH YOUR SUPPLER: Authorized by: Date: 1


In [24]:
from jiwer import wer, cer

print("Fine-tuned EasyOCR WER:", wer(ground_truth_clean, easyocr_ft_text_clean))
print("Fine-tuned EasyOCR CER:", cer(ground_truth_clean, easyocr_ft_text_clean))


Fine-tuned EasyOCR WER: 43.0
Fine-tuned EasyOCR CER: 33.07692307692308
